In [ ]:
!pip install mlflow

In [ ]:
!pip install awscli

  Using cached awscli-1.44.34-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9.1
    Uninstalling rsa-4.9.1:
      Successfully uninstalled rsa-4.9.1
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
# 1. Install MLflow
!pip install mlflow --quiet

import mlflow
from google.colab import output

# 2. Log your experiment
# Note: In Colab, it's safer to log to local files ("file:./mlruns") rather than
# starting a background HTTP server just to log data. The UI will read these files later.
mlflow.set_tracking_uri("file:./mlruns")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.3)
    print("Run completed and logged locally.")

# 3. Run the MLflow UI in the background
# We run this as a system command with '&' to keep it running in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

# 4. Expose the UI through Google Colab's reverse proxy
print("Click the link below to view the MLflow UI:")
output.serve_kernel_port_as_window(5001)

Run completed and logged locally.
Click the link below to view the MLflow UI:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df = df[~(df.clean_comment.str.strip() == "")]

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
def preprocess_comment(comment):

  #Convert to lowercase
  comment = comment.lower()

  #Rmove the trailing and leading whitespaces
  comment = comment.strip()

  #Remove newline character
  comment = re.sub('\n',' ',comment)

  #Remove non-alphanumeric character except pucntuation
  comment = re.sub(r'[^a-zA-Z0-9\s!?.,]','',comment)

  #Remove stopwords but retain the important ones
  stop_words = set(stopwords.words('english')) - {'not','no','but','however','yet'}
  comment = ' '.join([word for word in comment.split() if word not in stop_words])

  #Lemmatize the
  lemmatizer = WordNetLemmatizer()
  comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

  return comment

In [ ]:
#Apply the preprocessing step to clean the commment
df['clean_comment'] = df.clean_comment.apply(preprocess_comment)

In [ ]:
df.head()

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# step1 Vectorize the comment by bow
vectorizer = CountVectorizer(max_features=10000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_comment, df.category, test_size=0.2, random_state=42)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Set or create an experiment
mlflow.set_experiment("RF Baseline")

In [ ]:
with mlflow.start_run() as run:
  #Log a description for a run
  mlflow.set_tag("mlflow.runName", "RandomForest_Baseline_TrainTestSplit")
  mlflow.set_tag('experiment_type','baseline')
  mlflow.set_tag('model_type','RandomForestClassifier')

  #Add a description
  mlflow.set_tag("description", "Baseline RandomForest model for sentiment analysis using Bag of Words (BoW) with a simple train-test split")

  #Log parameter for vectorizer
  mlflow.log_param("vectorizer_type",'CountVectorizer')
  mlflow.log_param("vecorizer_max_features",vectorizer.max_features)

  # Log Random Forest parameters
  n_estimators = 200
  max_depth = 15

  mlflow.log_param("n_estimators", n_estimators)
  mlflow.log_param("max_depth", max_depth)

  # Initialize and train the model
  model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
  model.fit(X_train_vec, y_train)

  # Make predictions on the test set
  y_pred = model.predict(X_test_vec)

  # Log metrics for each class and accuracy
  accuracy = accuracy_score(y_test, y_pred)
  mlflow.log_metric("accuracy", accuracy)

  classification_rep = classification_report(y_test, y_pred, output_dict=True)

  for label, metrics in classification_rep.items():
    if isinstance(metrics, dict):
      for metric_name, metric_value in metrics.items():
        mlflow.log_metric(f"{label}_{metric_name}",metric_value)

  # Confusion matrix plot
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(8, 6))
  sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
  plt.xlabel("Predicted")
  plt.ylabel("Actual")
  plt.title("Confusion Matrix")

  # Save and log the confusion matrix plot
  plt.savefig("confusion_matrix.png")
  mlflow.log_artifact("/content/confusion_matrix.png")

  # Log the Random Forest model
  mlflow.sklearn.log_model(model, "random_forest_model")

  # Optionally log the dataset itself (if it's small enough)
  df.to_csv("dataset.csv", index=False)
  mlflow.log_artifact("/content/dataset.csv")

# Display final accuracy
print(f"Accuracy: {accuracy}")

In [ ]:
print(classification_report(y_test, y_pred))